<a href="https://colab.research.google.com/github/violetaElCactus/predictGames/blob/main/Metacritic_Inteligencia_artificial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importar librerías

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

print("Tensorflow: " + tf.__version__)

Tensorflow: 2.5.0


#Importar los datos

In [2]:
from google.colab import drive
drive.mount('/content/drive')
data_juegos = pd.read_csv("drive/My Drive/Colab Notebooks/metacritic_game_info.csv")
data_comentarios = pd.read_csv("drive/My Drive/Colab Notebooks/metacritic_game_user_comments.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Información juegos

In [3]:
data_juegos.head(10)

,Unnamed: 0,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players
0,0,The Legend of Zelda: Ocarina of Time,1998,Nintendo,Action Adventure;Fantasy,Nintendo64,99,9.1,1 Player
1,1,Tony Hawk's Pro Skater 2,2000,NeversoftEntertainment,Sports;Alternative;Skateboarding,PlayStation,98,7.4,1-2
2,2,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,PlayStation3,98,7.5,1 Player
3,3,SoulCalibur,1999,Namco,Action;Fighting;3D,Dreamcast,98,8.6,1-2
4,4,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,Xbox360,98,7.9,1 Player
5,5,Super Mario Galaxy,2007,Nintendo,Action;Platformer;Platformer;3D;3D,Wii,97,9.0,No Online Multiplayer
6,6,Super Mario Galaxy 2,2010,NintendoEADTokyo,Action;Platformer;Platformer;3D;3D,Wii,97,9.1,No Online Multiplayer
7,7,Grand Theft Auto V,2014,RockstarNorth,Action Adventure;Modern;Open-World,XboxOne,97,7.8,Up to 30
8,8,Grand Theft Auto V,2013,RockstarNorth,Modern;Action Adventure;Modern;Open-World,PlayStation3,97,8.3,Up to 16
9,9,Grand Theft Auto V,2013,RockstarNorth,Modern;Action Adventure;Modern;Open-World,Xbox360,97,8.3,Up to 16


##Información comentarios

In [4]:
data_comentarios.head(10)

,Unnamed: 0,Title,Platform,Userscore,Comment,Username
0,0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,"Everything in OoT is so near at perfection, it...",SirCaestus
1,1,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I won't bore you with what everyone is already...,Kaistlin
2,2,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Anyone who gives the masterpiece below a 7 or ...,Jacody
3,3,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I'm one of those people who think that this is...,doodlerman
4,4,The Legend of Zelda: Ocarina of Time,Nintendo64,10,This game is the highest rated game on Metacr...,StevenA
5,5,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I think it's funny that you have Zelda haters ...,joei1382
6,6,The Legend of Zelda: Ocarina of Time,Nintendo64,9,I played A Link To The Past so many times in m...,Corvix
7,7,The Legend of Zelda: Ocarina of Time,Nintendo64,10,The Legend of Zelda: Ocarina of Time is withou...,pittsburghboy91
8,8,The Legend of Zelda: Ocarina of Time,Nintendo64,10,"This review contains spoilers, cli...",Nosidda89
9,9,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I'm not kidding when I say that this is the on...,Regeneration13


In [5]:
data_comentarios = data_comentarios[data_comentarios['Comment'].str.len() > 280] #Comentario más largo que un Tweet.


In [6]:
data_comentarios = data_comentarios[data_comentarios['Comment'].map(lambda x: x.isascii())]


In [7]:
data_comentarios["Comment"] = data_comentarios["Comment"].str.lower()


In [8]:
df1 = data_comentarios.iloc[:60000]

In [9]:
len(df1)

60000

In [10]:
df1.drop(['Unnamed: 0','Username','Title','Platform'], axis=1, inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
df1.head(10)


,Userscore,Comment
2,10,anyone who gives the masterpiece below a 7 or ...
3,10,i'm one of those people who think that this is...
4,10,this game is the highest rated game on metacr...
5,10,i think it's funny that you have zelda haters ...
7,10,the legend of zelda: ocarina of time is withou...
8,10,"this review contains spoilers, cli..."
9,10,i'm not kidding when i say that this is the on...
10,10,"essentially one of the best games of all time,..."
11,10,this is the perfect game ! no doubt. the gamep...
16,10,"amazing game in every aspect, i just want to ..."


#Dividir los datos en entrenamiento, validación y prueba




In [12]:
train, validate, test = np.split(df1.sample(frac=1), [int(.7*len(df1)), int(.8*len(df1))])


In [13]:
train_comments = train['Comment']
train_labels = train['Userscore']
validate_comments = validate['Comment']
validate_labels = validate['Userscore']
test_comments = test['Comment']
test_labels = test['Userscore']

#Tokenizer


In [14]:
import keras
from keras.preprocessing.text import Tokenizer


In [15]:
vocab_size = 12000
tokenize = Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(train_comments)
sequences = tokenize.texts_to_sequences(train_comments)

In [17]:
matrix = tokenize.texts_to_matrix(train_comments, mode = 'binary')

In [18]:
max_seq_length = 280
sequences = keras.preprocessing.sequence.pad_sequences(
        sequences, maxlen=max_seq_length, padding="post"
    )

#Modelo

In [20]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

In [21]:
y_train_vec = to_categorical(train_labels, num_classes=11)
y_validate_vec = to_categorical(validate_labels,num_classes=11)
y_test_vec = to_categorical(test_labels,num_classes=11)

In [48]:
modelMtx = models.Sequential()
modelMtx.add(layers.Dense(256,activation='relu',input_shape=(12000,)))
modelMtx.add(layers.Dense(11, activation='softmax'))
modelMtx.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [49]:
modelMtx.fit(matrix, y_train_vec, epochs=4)

Epoch 1/4
1313/1313 [==============================] - 35s 26ms/step - loss: 0.0530 - accuracy: 0.5351
Epoch 2/4
1313/1313 [==============================] - 34s 26ms/step - loss: 0.0415 - accuracy: 0.6556
Epoch 3/4
1313/1313 [==============================] - 34s 26ms/step - loss: 0.0266 - accuracy: 0.8010
Epoch 4/4
1313/1313 [==============================] - 34s 26ms/step - loss: 0.0138 - accuracy: 0.9046


#Prueba

In [50]:
vocab_size = 12000
tokenize = Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(train_comments)
x_test_vec = tokenize.texts_to_sequences(test_comments)

In [51]:
x_test_vec_mtx = tokenize.texts_to_matrix(test_comments)

In [52]:
x_test_vec = keras.preprocessing.sequence.pad_sequences(
        x_test_vec, maxlen=max_seq_length, padding="post"
    )

In [32]:

#predictions = model.predict(x_test_vec)

In [53]:
predictionsMtx = modelMtx.predict(x_test_vec_mtx)

In [58]:
np.argmax(predictionsMtx[7459])

9

In [59]:
test_labels.iloc[7459]

9

In [47]:
test_comments.iloc[7457]

" first off, if your a reviewer, don't give a game a perfect score but then say it is almost perfect, give it a 9. to rate and compare this game one must look at other games of this type, hack and slash platformer! comparisons should be done with similar games such as the very under developed dante. if only dante would have been done on this grand of a scale!! most of the game is first off, if your a reviewer, don't give a game a perfect score but then say it is almost perfect, give it a 9. to rate and compare this game one must look at other games of this type, hack and slash platformer! comparisons should be done with similar games such as the very under developed dante. if only dante would have been done on this grand of a scale!! most of the game is action platforming with very few moments of just holding the controller and doing nothing. the cut scenes feel like they are in the appropriate places and don't interrupt the flow of the game. the quick time events seem appropriate with